# Estimating Exoplanet Occurrence Rates: Survey Sensitivity, Detection Efficiency, and Statistical Modeling

---

### Purpose and Content
This notebook is focused on the statistical analysis of exoplanet detection rates, specifically for a simulated or real survey. The analysis involves:
* **Reading and processing a dataset** of detected planetary events, including their properties such as mass, semi-major axis, and detection weights.
* **Applying selection cuts** to the data to filter for events that meet certain criteria (e.g., signal-to-noise, in-season detection).
* **Correcting for survey geometry and false positive rates** using external data (e.g., covering factors).
* **Calculating and visualizing detection efficiencies** as a function of planet mass and semi-major axis.
* **Building and plotting histograms** of key parameters (e.g., semi-major axis, event times, impact parameters).
* **Computing the survey sensitivity** and the expected number of detections for different planet populations.
* **Fitting occurrence rate models** (e.g., power laws in mass and semi-major axis) to the observed planet sample using likelihood and MCMC methods.
* **Visualizing the results** with various plots, including detection efficiency maps and parameter distributions.

### Key Methods and Libraries
* **Python scientific stack:** pandas, numpy, matplotlib, scipy, lmfit, emcee, corner.
* **Statistical modeling:** Maximum likelihood and MCMC for fitting planet occurrence rates.
* **Survey simulation:** Functions to draw samples, apply weights, and correct for observational biases.

### Scientific Context

This notebook implements a standard pipeline for calculating exoplanet occurrence rates from a microlensing survey, heavily influenced by methods from [Suzuki et al. (2016)](https://ui.adsabs.harvard.edu/abs/2016ApJ...833..145S/abstract) and [Penny et al. (2019)](https://ui.adsabs.harvard.edu/abs/2019ApJS..241....3P/abstract).

The core idea is to take a simulated dataset of planetary events, which have been corrected for observational biases like seasonal gaps, and determine the underlying true distribution of planets. We do this by:
1.  **Filtering** the raw simulation output to create clean samples of all possible events (`dat`) and only those actually detected (`det`).
2.  **Calculating the survey's detection efficiency**, which creates a map of how sensitive the survey is to planets of different masses (`m_p`) and semi-major axes (`a`).
3.  **Defining an occurrence rate model** (e.g., a power law) that describes the planet distribution.
4.  **Fitting this model** to the detected planets using a Maximum Likelihood Estimate (MLE) and exploring the parameter space with a Markov Chain Monte Carlo (MCMC) to find the best-fit parameters and their uncertainties.

The final goal is to produce a measurement like `η_Earth` (eta-Earth), the fraction of stars that host an Earth-like planet.

### Outputs

* Plots of the spatial and parameter distributions of detected events.
* Detection efficiency maps in planet mass and semi-major axis space.
* Calculation of the total number of expected events after corrections.
* Preparation for or execution of statistical fits to infer underlying planet population parameters.

### Overview

This notebook is a comprehensive analysis pipeline for estimating exoplanet occurrence rates from a survey,  including data processing, bias correction, statistical modeling, and visualization. It is for used in the context of a microlensing survey simulation or data analysis, with a focus on understanding the true distribution of exoplanets after accounting for observational biases.

### Known Issues

There is an unverified `scalefac ` in section ["Applying the Survey Covering Factor and Final Weight Corrections"](#applying-the-survey-covering-factor-and-final-weight-corrections). 
Without this variable to provide meaningful normalization, this notebook is only useful for pipeline validation.
`scalefac` converts "weight per simulated star" to "planets per real star"

It requires knowing:

* Simulated stellar density (stars/sq.deg in simulation)
* Actual Galactic stellar density (stars/sq.deg in reality)

Without this, occurrence rates remain relative but not absolute

## How to Use This Notebook (The General Case)

---

This notebook is designed to guide you through the process of analyzing microlensing exoplanet survey data, estimating occurrence rates, and understanding your survey’s sensitivity and results.  
It is structured to be beginner-friendly, with clear explanations and example outputs at each step.

### Step-by-Step Instructions

1. **Set Up Your Environment**
   - Make sure you have all the required Python packages installed (see the `environment.yml` or `requirements.txt`).
   - Place your data files (e.g., HDF5 event files, covering factor files) in the correct locations as specified in the code cells.

2. **Load and Prepare Your Data**
   - Run the cells that load your planet event data and apply any necessary preprocessing.
   - The notebook will add extra columns (like insolation and season flags) to your data for later analysis.

3. **Apply Survey Corrections**
   - The notebook will automatically correct for survey coverage, false positives, and other effects to ensure your results are fair and accurate.

4. **Filter and Visualize Your Data**
   - Use the provided cells to filter your data based on detection criteria.
   - Visualize the distribution of your events in various ways (sky position, distance, time, impact parameter, etc.).
   - Example plots are provided so you can check if your outputs look reasonable.

5. **Calculate Survey Sensitivity**
   - The notebook computes how sensitive your survey is to different types of planets (by mass, distance, and insolation).
   - Visualize these sensitivities with heatmaps and summary plots.

6. **Fit Occurrence Rate Models**
   - Use maximum likelihood estimation (MLE) to fit your occurrence rate model to the data.
   - Check the fit by comparing model predictions to observed distributions.

7. **Run MCMC for Uncertainties**
   - Set up and run Markov Chain Monte Carlo (MCMC) to explore the full range of model parameters that fit your data.
   - Use the corner plots and model curve plots to check for convergence and parameter correlations.

8. **Interpret Your Results**
   - Use the best-fit parameters and their uncertainties to draw scientific conclusions about planet populations in your survey.
   - Compare your results to example outputs and literature values as a sanity check.

### Tips for Success

- **Read the markdown explanations** in each section—they’re written to be insultingly accessible even if you’re new to this kind of analysis.
- **Check the example outputs** to make sure your results look reasonable at each step.
- **Use the function links** (e.g., [`ssn`](#what-does-ssn-do)) to jump to detailed explanations of what each function does.
- **If you get stuck or see unexpected results**, review the diagnostic plots and pro tips, or consult the documentation for the relevant packages (like [`emcee`](https://emcee.readthedocs.io/en/stable/)).

### Customization

- You can change the data file paths, parameter ranges, and model forms to suit your own survey or science goals.
- If you want to analyze a different dataset, just update the relevant file paths and rerun the notebook.

## How to Use this Notebook to Check Your Simulation

---

You made the simulation, so you know the "true" occurrence rate parameters that you programmed in. The entire point of running this notebook is to see if the analysis can correctly figure out those parameters from the data you generated.

Follow these steps:

1.  **Set Your Input File:** In the [**"Loading Your Planet Data"** section](#loading-your-planet-data), change the `filename` variable to point to your Gulls simulation output file.
    ```python
    # Change this line to your file:
    filename = '/path/to/your/gulls_simulation_output.hdf5'
    ```

2.  **Enter YOUR "True" Parameters:** This is the most important step. In the section [**"Setting Up True Model Parameters for Testing,"**](#setting-up-true-model-parameters-for-testing) find the `true_params` dictionary. You **must** update the values for `logC_true`, `n_true`, and `m_true` to match the parameters you used to create your simulation data.

    ```python
    # UPDATE THESE VALUES:
    n_true = -0.9      # Whatever your true mass slope was
    m_true = 0.5       # Whatever your true SMA slope was
    logC_true = np.log10(1.5)  # Whatever your true normalization was
    
    true_params = {'logC':logC_true,
                   'n':n_true,
                   'm':m_true}
    ```

3.  **Run the Notebook:** Run all the cells from top to bottom.

4.  **Check the Results:** Go to the [final plots](#visualizing-mcmc-results-with-a-corner-plot):
    * **In the distribution plots:** The "Fit Rate" line should land right on top of the "True Rate" line. If it doesn't, something is wrong.
    * **In the corner plot:** The blue lines showing the "truths" should go right through the middle of your data blobs (the posteriors). This proves the MCMC found the right answer.

This turns the notebook from a simple analysis tool into a validation tool. It's no longer just giving you an answer; it's showing you if the answer is *correct*.

## How this Notebook Relates to the Roman Science Requrements and What Needs Doing.

---

> **RST Level-1 Science Requirements** - Those highlighted are not verified at all. Those verified do so at \~20% margin with old slew/settle times and assuming 6 x 72 day seasons.
> * **EML 2.0.1:** RST shall be capable of measuring the mass function of exoplanets with masses in the range $1M_{\oplus} < m < 30M_{\rm{Jupiter}}$ and orbital semi-major axes $\ge 1 \rm{au}$ to better than 15% per decade in mass.
> * **EML 2.0.2:** RST shall be capable of measuring the frequency of bound exoplanets with masses in the range $0.1M_{\oplus} < m < 0.3M_{\oplus}$ to better than 25%.
> * [**EML 2.0.3:** RST shall be capable of determining the masses of, and distances to, host stars of 40% of the detected planets with a precision of 20% or better.]()
> * **EML 2.0.4:** RST shall be capable of measuring the frequency of free floating planetary- mass objects in the Galaxy from Mars to 10 Jupiter masses. If there is $1M_{\oplus}$ free-floating planet per star, measure this frequency to better than 25%.
> * [**EML 2.0.5:** RST shall be capable of estimating $\eta_{\oplus}$ (the frequency of planets orbiting FGK stars with mass ratio and estimated projected semimajor axis within 20% of the Earth-Sun system) to a precision of 0.2 dex via extrapolation from larger and longer-period planets.]()

The whole point of this exercise is to check off those science requirements for the Roman telescope. The notebook is just the tool. You need to connect the dots.

Here's the breakdown on how this notebook helps you tackle that list.

### How This Notebook Answers Your Science Questions:

#### 1\. For Measuring the Planet Mass Function (Requirement EML 2.0.1)

  * **What it is:** This is the main job of the notebook. The goal is to figure out how common planets of different masses are.
  * **What you needs to do:**
    1.  Run the entire notebook to get a sample of *detected* planets.
    2.  Pay close attention to the **"Modelling the Occurance Rates"** and **"Re-fitting with MCMC"** sections. The MCMC fit gives the best-fit parameters for the planet population, specifically the power-law slope for mass (the parameter `n`).
    3.  The final corner plot shows the measurement and its uncertainty. This directly addresses the "better than 15% per decade in mass" part. The plots in **"Occurance Rate Fitting Results"** are her proof that the model fits.

#### 2\. For Finding the Frequency of Low-Mass Planets (Requirement EML 2.0.2)

  * **What it is:** This is just a specific slice of the big mass function from the first requirement.
  * **What you needs to do:**
    1.  Once you have the final, best-fit model from the MCMC run, you needs to integrate that function over the specified mass range ($0.1 M\_{\\oplus}$ to $0.3 M\_{\\oplus}$).
    2.  This isn't a pre-built cell, so you'll have to add a little bit of code at the end to do this calculation using the MCMC results. It's a direct outcome of the main analysis.

#### 3\. For Estimating η⊕ (eta-Earth) (Requirement EML 2.0.5)

  * **What it is:** This is the big one, the reason for the whole project. It's about finding Earth-like planets.
  * **What you needs to do:**
    1.  Just like with the low-mass planets, this requires taking the final fitted model from the MCMC.
    2.  You'll need to integrate that model over the specific "Earth-like" box—that means filtering by planet mass *and* by **insolation** (how much light the planet gets from its star).
    3.  The notebook already calculates insolation for you in the [**"Adding Extra Information to Your Data"** cell](#adding-extra-information-to-your-data-insolation) and even makes a `mass-insolation` detection map. You have all the pieces; you just needs to put them together in a final calculation. The [**"How to Interpret the Results"** section](#how-to-interpret-the-results) points this out.

#### 4\. What This Notebook *Doesn't* Do (But Is Still Needed For)

This part is crucial, so you doesn't waste your time looking for things that aren't there.

  * **Host Star Masses (EML 2.0.3):** This notebook simulates the *detection* of planets, not the detailed follow-up measurements needed to determine the mass of the host stars. As the [Penny (2019)](https://ui.adsabs.harvard.edu/abs/2019ApJS..241....3P/abstract) paper explains, that requires measuring things like the lens flux or parallax over the course of the mission. This notebook's output—the list of detected planets—is the *input* for that separate analysis.
  * **Free-Floating Planets (EML 2.0.4):** This analysis is explicitly for *bound* planets. [Penny (2019)](https://ui.adsabs.harvard.edu/abs/2019ApJS..241....3P/abstract) even says that free-floating planets are a different challenge that requires a separate paper. The method is similar, but the models and event selection would be different. You can't use this notebook for that requirement without significant changes and completely different simulation data.

### The Key: Comparing Your Uncertainties to the Requirements

The most important result from this notebook is not the best-fit value for a parameter—it's the **uncertainty** on that value. The whole point of this exercise is to see if the final error bars from your MCMC fit are small enough to satisfy the requirements.

Think of it like this:

* The requirement says: "measure the mass function... to **better than 15%**" (EML 2.0.1).
* Your MCMC will give you a result for the mass slope, something like: `n = -0.9 ± 0.1`.
* That `± 0.1` is your uncertainty. In this case, it's about an 11% measurement ($0.1 / 0.9 \approx 11\%$).
* Since 11% is better than 15%, you've met the requirement. If your uncertainty was `± 0.2`, you would have failed.

**Every requirement is a test of the *precision* of your final MCMC results.** Your job is to prove that this pipeline can produce measurements that are not just *right*, but *certain enough*.

## Installation

---

 This notebook requires several Python packages that can be installed using conda or pip.
 
 Using conda (recommended):
 ```bash
 conda env create -f env.yml
 conda activate eta-earth-analysis
 ```
 
 Using pip:
 ```bash
 pip install -r requiremenets.txt
 ```
 
 The required packages are:
 - `pandas`: For data manipulation and analysis
 - `numpy`: For numerical computing
 - `matplotlib`: For plotting
 - `scipy`: For scientific computing
 - `lmfit`: For curve fitting and parameter estimation
 - `emcee`: For MCMC sampling
 - `corner`: For visualization of MCMC results
 - `jupyterlab`: For running this notebook
 - `ipykernel`: For Jupyter notebook support
 - `h5py`: For HDF5 file support

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import scipy.integrate
import numpy as np
import matplotlib
import lmfit
import emcee
import corner
import timeit
import os
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

## General Functions

---

### What does `draw_sample` do?

Imagine you have a big list of events (like a list of planets, or lottery tickets). Each event has a “weight”—think of this as how likely or important that event is. Some events are more likely to be picked than others.
This function helps you make a new list by randomly picking events from your big list. But it doesn’t pick them all with the same chance:

* Events with a bigger weight are more likely to be picked.
* Events with a smaller weight are less likely to be picked.

#### Here’s how it works, step by step:

1. **Add up all the weights.** This gives you a sense of how many events you “expect” to pick.
2. **Decide how many to pick.** The function uses a bit of randomness (like rolling dice) to decide the actual number, based on the total weight.
3. **Pick events, one by one.** Each time, it’s more likely to pick an event with a bigger weight.
4. **Make a new list.** The result is a new list of events, picked in a way that respects their weights.

> **Why do this?**
>
> This is useful when you want to simulate what you might see in a real experiment, where some things are more likely to happen than others.

In [5]:
# function to draw a sample of events 
def draw_sample(df,weight_col='fwc'):
    """
    Draw a Poisson-resampled subset of events using per-event weights.
    
    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame containing event information.
    weight_col : str, optional
        Name of the column in ``df`` containing the weights. These
        weights are normalized and used as the selection probability.
    
    Returns
    -------
    pandas.DataFrame
        Random sample of events drawn with replacement according to the
        provided weights.
    """
    # assumes weights are correct
    # norm is the sum of weights (i.e., expected number of detections)
    norm = np.sum(df[weight_col])
    # probabilities for individual events are equal to normalized weights
    probs = df[weight_col] / norm
    # the number of events drawn is a poisson draw of the expected number of events
    ndraw = np.random.poisson(lam=norm)
    # draw a random subset of events to be in the sample. Should replace be true?
    sample_idx = np.random.choice(df.shape[0],ndraw,replace=True,p=probs)
    # generate a data frame of the sample from the original data frame                                                                                                                                                                                                
    sample = df.iloc[sample_idx]
    return sample

### What does `ssn` do?

Imagine you have a list of events, and each event happened at a certain time (let’s call it `x`). You want to know if that time falls inside a “season” (a special time period), or outside of it.

This function takes a time (or a list of times) and tells you:

* **If the time is inside a season:** It gives you a positive number (which season it is).
* **If the time is outside a season:** It gives you a negative number (which season it’s after).

#### How does it work?

* It checks the value of x and compares it to a bunch of cut-off numbers.
* Depending on where x falls, it returns a number that tells you if it’s in a season (positive) or not (negative), and which one.

> **Why do this?**
>
> This is useful if you only care about events that happen during certain times (seasons), or if you want to filter out events that happen outside those times.

> **In short:**
>
> You give it a time, it tells you if that time is in a special “season” or not, and which one.

In [6]:
def ssn(x):
    """
    Determine whether a given time falls within an observing season.
    
    Parameters
    ----------
    x : array_like or float
        Time(s) in days to evaluate.
    
    Returns
    -------
    int or ndarray
        Positive values denote the season number if ``x`` is in season.
        Negative values indicate the preceding season when out of season.
    """
        if np.isscalar(x):
                if x<112.5:
                        return 0
                if x<=184.5:
                        return 1
                if x<292.25:
                        return -1
                if x<=364.25:
                        return 2
                if x<477.75:
                        return -2
                if x<=549.75:
                        return 3
                if x<1388.0:
                        return -3
                if x<=1460.0:
                        return 4
                if x<1573.5:
                        return -4
                if x<=1645.5:
                        return 5
                if x<1753.25:
                        return -5
                if x<=1825.25:
                        return 6
                return -6
        else:
                ret = x*0+99
                ret[(ret==99) * (x<112.5)] = 0
                ret[(ret==99) * (x<=184.5)] = 1
                ret[(ret==99) * (x<292.25)] = -1
                ret[(ret==99) * (x<=364.25)] = 2
                ret[(ret==99) * (x<477.75)] = -2
                ret[(ret==99) * (x<=549.75)] = 3
                ret[(ret==99) * (x<1388.0)] = -3
                ret[(ret==99) * (x<=1460.0)] = 4
                ret[(ret==99) * (x<1573.5)] = -4
                ret[(ret==99) * (x<=1645.5)] = 5
                ret[(ret==99) * (x<1753.25)] = -5
                ret[(ret==99) * (x<=1825.25)] = 6
                ret[ret==99] = -6

                return ret

### What does `FPF` do?

Sometimes, when you find something (like a planet), it might actually be a “false positive”—meaning it looks real, but it’s not. This function helps you figure out how likely it is that a planet you found is actually a mistake (a false positive).

#### How does it work?

* It looks at the mass of the planet.
* It uses a special formula (from a scientific paper) to calculate the chance that the planet is a false positive.
* The answer is a number between 0 and 1. A bigger number means more likely to be a false positive; a smaller number means less likely.

> **Why do this?**
>
> This helps you correct your results, so you don’t count things that are probably not real planets.

> **In short:**
>
> You give it a planet’s mass, it tells you how likely it is that planet is actually just a mistake.

In [7]:
def FPF(data,mearth,mass_col=42):
    """
    Apply the false positive fraction correction from Penny (2019).
    
    Parameters
    ----------
    data : pandas.DataFrame
        Planet catalog containing a ``Planet_mass`` column.
    mearth : float
        Mass of Earth in Solar masses.
    mass_col : int, optional
        Unused placeholder kept for legacy reasons.
    
    Returns
    -------
    ndarray
        False positive fraction for each planet.
    """
    alpha = 1.25
    beta = 2.7
    return (1+10**(alpha*np.log10(data['Planet_mass']/mearth)+beta))**-1

### What does `mass_func_arr` do?

Suppose you want to know how common different kinds of planets are—like, are big planets more common than small ones? Are planets far from their star more common than close ones? This function helps you figure that out.

#### How does it work?

* You give it a list of planets, and some numbers (called “parameters”) that describe how you think planet types are spread out.
* For each planet, it looks at its mass and how far it is from its star.
* It uses a math formula (called a “power law”) with your parameters to calculate a number for each planet. This number tells you how likely that kind of planet is, according to your model.
  $$ \frac{d^2 N}{d\log m_p  d\log a} = C \cdot \left(\frac{m_p}{M_\oplus}\right)^n \cdot \left(\frac{a}{\text{au}}\right)^m $$

> **Why do this?**
>
> This helps you build a picture of what kinds of planets are most common, based on their mass and distance from their star.

> **In short:**
>
> You give it a bunch of planets and some model settings, and it tells you how common each planet type is, using a math formula.

In [8]:
def mass_func_arr(pars,planet_sample,mearth=3e-6):
    """
    Evaluate a power-law occurrence model for a set of planets.
    
    Parameters
    ----------
    pars : dict or sequence
        Model parameters ``{'logC', 'n', 'm'}`` or ``[logC, n, m]``.
    planet_sample : pandas.DataFrame or array_like
        Planet properties or an array of log masses.
    mearth : float, optional
        Mass of Earth in Solar masses.
    
    Returns
    -------
    ndarray
        Occurrence rate evaluated at each planet.
    """
    # compute the mass function values for a set of pars given a dataframe of events
    # this version is for a saturated power law in mass, single power law in SMA
    #logC, logMbreak, n, m = theta 
    try:
        M_planet = planet_sample['Planet_mass']/mearth
        sma = planet_sample['Planet_semimajoraxis']
    # this is to catch when an array of values (instead of data frame) is an argument, could be smarter
    except:
        M_planet = 10**planet_sample
        sma = np.ones(len(M_planet))
    
    # set up return array
    ret = np.ones(len(sma))
    
    
    # unpack the pars into a dict if in list form
    if type(pars) is list or type(pars) is np.ndarray:
        #starting_mean = np.array([logC1_start,logMbreak_start,logC2_start,logMS2_start,n2_start,m_start])                                                                                                             
        theta = {'logC':pars[0],
                 'n':pars[1],
                 'm':pars[2]}
    # else pars is already a dictionary
    else:
        theta=pars

    # compute rates
    ret = 10**theta['logC'] * (M_planet)**theta['n'] * sma**theta['m']
    return ret

### What does `occ_arr` do?
Let’s say you want to make a map that shows how common different types of planets are, depending on their mass (how heavy they are) and their distance from their star. This function helps you fill in that map.

#### How does it work?

* You give it a grid of possible planet masses and distances (think of a big table with all the combinations).
* You also give it some numbers (parameters) that describe your guess for how planet types are spread out.
* For every spot in the grid (every combination of mass and distance), it uses a math formula to figure out how common a planet would be there, according to your model.

> **Why do this?**
>
> This lets you see, all at once, which kinds of planets your model says should be common or rare, across a whole range of masses and distances.

> **In short:**
> 
> You give it a grid of planet types and some model settings, and it fills in the grid with numbers that say how common each type should be.

In [9]:
def occ_arr(pars,m,a,mearth=3e-6):
    """
    Compute the occurrence rate on a mass--semi-major axis grid.
    
    Parameters
    ----------
    pars : dict or sequence
        Model parameters as in :func:`mass_func_arr`.
    m : ndarray
        log10 planet masses.
    a : ndarray
        log10 semi-major axes.
    mearth : float, optional
        Mass of Earth in Solar masses.
    
    Returns
    -------
    ndarray
        2D array of occurrence rates with shape ``(len(a), len(m))``.
    """
    #logC, logMbreak, n, m = theta
    # delog the grid point values for mass func calcs
    M_planet = 10**m
    sma = 10**a
    # just making some arrays of ones
    m_vals = (M_planet/M_planet).copy() 
    s_vals = (sma/sma).copy()
    
    # unpack the pars into a dict if in list form
    if type(pars) is list or type(pars) is np.ndarray:
        theta = {'logC':pars[0],
                 'n':pars[1],
                 'm':pars[2]}                                                                                                                                                                                         
    else:
        theta=pars

    m_vals = 10**theta['logC'] * (M_planet)**theta['n']
    s_vals = sma**theta['m']
    return m_vals.reshape(1,-1)*s_vals.reshape(-1,1)

## MCMC Functions

---

### What does `N_exp_real` do?

Imagine you want to know: **“If my model is right, how many planets should I expect to actually find in my survey?”**

This function helps you figure that out.

#### How does it work?

It takes your model (the “theta” numbers) and a map of how good your survey is at finding planets (the “survey sensitivity”). It makes a big grid of all possible planet types (different masses and distances). For each spot in the grid, it figures out how common that planet should be (using your model) and how likely your survey is to find it. It adds up all those numbers, across the whole grid, to get the total number of planets you’d expect to detect.

> **Why do this?**
>
> This tells you, for your model and your survey, how many planets you should see—so you can compare with what you actually found.

> **In short:** 
>
> You give it your model and your survey’s abilities, and it tells you how many planets you should expect to find.

In [10]:
def N_exp_real(theta,planet_sample,survey_sensitivity):
    """
    Integrate the survey sensitivity to get the expected number of detections.
    
    Parameters
    ----------
    theta : dict or sequence
        Occurrence rate parameters.
    planet_sample : pandas.DataFrame
        Sample of detected planets.
    survey_sensitivity : ndarray
        2D sensitivity grid.
    
    Returns
    -------
    float
        Expected number of detected planets.
    """
    # integrate the survey sensitivity to get the expected number of detections
    # these should be arguments, but left in for now
    a_low,a_high,m_low,m_high = 0.01,10.,0.01,10.
    nbins = survey_sensitivity.shape[0]+1
    # generate grid points to integrate over
    m_bins = np.linspace(np.log10(m_low),np.log10(m_high),nbins,endpoint=True)
    a_bins = np.linspace(np.log10(a_low),np.log10(a_high),nbins,endpoint=True)
    m_bin_centers = (m_bins[1:]+m_bins[:-1])/2
    a_bin_centers = (a_bins[1:]+a_bins[:-1])/2

    # make a 2d array of the expected number of detections given the occurrence rate parameters being proposed (theta)
    zz = occ_arr(theta,a_bin_centers.reshape(-1,1),m_bin_centers.reshape(1,-1))
    # integrate over the occurrence rate grid
    # the next line *SHOULD* work, it should just be a 2d integral. I haven't gotten a chance to retest it
    #int1 = scipy.integrate.simps([scipy.integrate.simps(zz_x,a_bin_centers) for zz_x in zz*survey_sensitivity],m_bin_centers)
    # split integral into two parts to clarify/test
    inner_int = np.array([scipy.integrate.simpson(row,x=a_bin_centers) for row in zz*survey_sensitivity])
    int1 = scipy.integrate.simpson(inner_int,x=m_bin_centers)
    # this is a short cut to "integrate" I think, just occurrence rate * sensitivity * bin size
    # int1 = np.sum(zz*survey_sensitivity)*(m_bins[1]-m_bins[0])*(a_bins[1]-a_bins[0])
    return int1

### What does `N_exp_true` do?

This function gives you a quick and simple way to guess how many planets your survey should find, using your model.  
Instead of carefully checking how good your survey is at finding each type of planet, it just adds up the “weights” for all the planets you could have found, using your model and a simple multiplier.

It’s a rough estimate—faster, but not as accurate as the more careful method.

#### How does it work?

- It uses your model to calculate a number for each possible planet (using their mass and distance).
- It multiplies that number by each planet’s “weight” (how likely it is to be found).
- It multiplies by a simple constant (just a scaling factor).
- It adds up all those numbers to get the total expected number of planets.

> **Why do this?**
>
> This is useful if you want a quick answer or to test your code, but you should use the more careful method for real results.

> **In short:** 
>
> It’s a shortcut to estimate how many planets you’d expect to find, but it’s not as trustworthy as the main method.

In [11]:
def N_exp_true(theta,planet_sample,det_full):
    """
    Estimate the expected detections by summing weights.
    
    Parameters
    ----------
    theta : dict or sequence
        Occurrence rate parameters.
    planet_sample : pandas.DataFrame
        Planet sample.
    det_full : pandas.DataFrame
        Full detection catalog including weights.
    
    Returns
    -------
    float
        Sum of weighted occurrence values.
    """
    # "integrate" the expected number of detections by just summing weights. This is a too-accurate method,
    # and the other should be preferred. Made for testing, and would need to be updated. 
    raww_col = 'final_weight'
    int_value = 9
    weights = mass_func_arr(theta,det_full)*det_full[raww_col]*int_value
    #print("Nexp: ",np.sum(weights))                                                                                                                                                                                   
    return np.sum(weights)

### What does `ln_like` do?

This function helps you figure out how well your model matches the planets you actually found in your survey.  
It gives you a score (called the “log likelihood”)—a bigger score means your model fits the data better.
This score is the log-likelihood, based on Equation 6 from [Suzuki et al. (2016)](https://ui.adsabs.harvard.edu/abs/2016ApJ...833..145S/abstract).

#### How does it work?

- It uses your model to guess how many planets you should find (using `N_exp_real`).
- It checks how well your model matches the planets you actually found, using their properties and how easy they were to detect.
- It combines these numbers into a single score:  
  - If your model predicts the right number and types of planets, the score is higher.
  - If your model is way off, the score is lower.

> **Why do this?**
>
> This score helps you compare different models and find the one that best matches your data.

### What does `ln_like_neg` do?

This is just a helper function for computers.  
It takes the score from `ln_like` and flips it to a negative number.

#### How does it work?

- It runs `ln_like` and multiplies the answer by -1.

> **Why do this?**
>
> Some computer tools like to find the smallest number (minimize), so this makes it easier for them to find the best model.

In [12]:
def ln_like(theta, planet_sample,survey_sensitivity):  
    """
    Compute the log-likelihood for a set of parameters.
    
    Parameters
    ----------
    theta : dict or sequence
        Occurrence rate parameters.
    planet_sample : pandas.DataFrame
        Detected planet sample with sensitivity values.
    survey_sensitivity : ndarray
        Survey sensitivity map.
    
    Returns
    -------
    float
        Natural logarithm of the likelihood.
    """
    #logA, logB, logM_br, logM_slope, n, p, m = theta 
    # evaluate the mass function at all planets in the sample  
    mass_func_vals = mass_func_arr(theta,planet_sample)
    # determine the number of expected detections
    # this should integrate over the survey sensitvity
    N_exp_1 = N_exp_real(theta,planet_sample,survey_sensitivity)
    # this version numerically determines the expected number of planets by just summing weights
    #N_exp_2 = N_exp_true(theta,planet_sample,sensitivity_sum,parameters_to_fit,det_full)                                                                                                                               
    #print('Nexp1, Nsamp :',N_exp_1, planet_sample.shape[0])
    # log likelihood summation. Recall that the individual sensitivities are stored in the first column, 
    # which is done in data processing.
    like_ = (-N_exp_1 + np.sum(np.log(mass_func_vals*planet_sample["sensitivity"])))
    
    return like_

# calculate the negative log likelihood for minimization
def ln_like_neg(theta, planet_sample,survey_sensitivity):
    """
    Negative log-likelihood for minimization routines.
    
    Parameters
    ----------
    theta : dict or sequence
        Occurrence rate parameters.
    planet_sample : pandas.DataFrame
        Detected planet sample.
    survey_sensitivity : ndarray
        Survey sensitivity map.
    
    Returns
    -------
    float
        Negative log-likelihood.
    """
    return -1*ln_like(theta, planet_sample,survey_sensitivity)

### What does `ln_prior` do?

This function checks if the numbers (parameters) you’re using for your model are reasonable.  
It helps make sure you don’t try out crazy or impossible values when testing different models.

#### How does it work?

- It looks at each parameter in your model and checks if it’s inside a “safe” range.
- If all the parameters are in the allowed range, it returns 0 (which means “okay!”).
- If any parameter is outside the allowed range, it returns a very bad score (−infinity), which tells the computer to ignore this set of parameters.

> **Why do this?**
>
> This keeps your model from trying out nonsense values and helps your results make sense.

In [13]:
def ln_prior(theta,paramters_to_fit=None):
    """
    Prior probability for the model parameters.
    
    Parameters
    ----------
    theta : sequence
        Array of model parameters ``[logC, n, m]``.
    paramters_to_fit : sequence, optional
        Placeholder for parameter selection (unused).
    
    Returns
    -------
    float
        ``0.0`` if parameters are within allowed ranges, ``-np.inf`` otherwise.
    """
    # prior ranges, right now just uniform priors. You will need to set these up for the 
    # variables being investigated, as well as suitable ranges
    if not np.log10(0.01) < theta[0] < np.log10(10):
        return -np.inf
    if not -2 < theta[1] < 2:
        return -np.inf
    if not -2 < theta[2] < 2:
        return -np.inf                                                                                                                                                                                       
    return 0.0

### What does `ln_prob` do?

This function checks how good your model is, using both the rules for what’s allowed (the “priors”) and how well it matches the data (the “likelihood”).  
It gives you a final score for your model—higher is better.

#### How does it work?

- It checks if your model’s parameters are allowed (using `ln_prior`). If not, it gives a very bad score (−infinity).
- It checks how well your model matches the data (using `ln_like`). If something goes wrong, it also gives a bad score.
- If everything is okay, it adds the two scores together to get the final answer.

> **Why do this?**
>
> This is the main score used to compare different models. It helps you find the model that both makes sense and fits your data best.

In [14]:
def ln_prob(theta, planet_sample, det_full):
    """
    Log-probability combining prior and likelihood.
    
    Parameters
    ----------
    theta : sequence
        Model parameters.
    planet_sample : pandas.DataFrame
        Detected planet sample.
    det_full : pandas.DataFrame
        Survey sensitivity or detection grid.
    
    Returns
    -------
    float
        Natural logarithm of the posterior probability.
    """
    # calculate the prior value given theta  
    ln_prior_ = ln_prior(theta) 
    # check that the prior did not go out of bounds; return negative infinity if it did
    if not np.isfinite(ln_prior_):
        return -np.inf
    # calculate the likelihood
    ln_like_ = ln_like(theta, planet_sample, det_full)

    # if for some reason likelihood is nan, return negative infinity, shouldn't happen and can probably be removed                                                                                                                                                                       
    if np.isnan(ln_like_):
        return -np.inf
    # multiply the prior and the lkikelihood
    ln_prob_ = ln_prior_ + ln_like_
    # return the probability, what emcee expects
    return ln_prob_ 

## Set-Up Functions

---

### What does `integrate_cassan` do?

This function helps you figure out how many planets you’d expect to find in a certain range of sizes (masses) and distances from their star, using a the [Cassan (2012)](https://ui.adsabs.harvard.edu/abs/2012Natur.481..167C/abstract) mass function.

#### How does it work?

- You give it a range for planet distance (`a_low` to `a_high`) and planet mass (`m_low` to `m_high`).
- It uses a special formula to calculate how many planets should be in that range.
- If the smallest mass is bigger than 5.2 Earth masses, it uses one formula.
- If the range includes smaller planets, it splits the calculation and adds up both parts.

> **Why do this?**
>
> This lets you estimate how many planets are in a certain part of your “planet map,” using a model that matches what scientists have found.

In [15]:
def integrate_cassan(a_low,a_high,m_low,m_high):
    """
    Integrate the modified Cassan (2012) mass function.
    
    Parameters
    ----------
    a_low, a_high : float
        Bounds on semi-major axis in au.
    m_low, m_high : float
        Bounds on planet mass in Earth masses.
    
    Returns
    -------
    float
        Integral of the Cassan mass function over the given range.
    """
    #earht masses                                                                                                                                                                                                      
    a_int = (np.log(a_high)-np.log(a_low))/np.log(10)

    if m_low > 5.2:
        m_int = -4.06524/(m_high**(0.74)) + 4.06524/(m_low**(0.74))
        return (a_int*m_int)
    else:
        m_int_low = 2*(np.log(5.2)-np.log(m_low))/np.log(10)
        m_int_high = -4.06524/(m_high**(0.74)) + 4.06524/(5.2**(0.74))
        return (m_int_low+m_int_high)*a_int

### What does `integrate_loguniform` do?

This function helps you figure out how many planets you’d expect to find in a certain range, if planets are spread out evenly on a log scale (meaning, each “step” in size or distance is just as likely as any other).

#### How does it work?

- You give it a range for planet distance (`a_low` to `a_high`) and planet mass (`m_low` to `m_high`).
- It calculates the area of that range on a log scale (using logarithms).
- The answer tells you how much of the “planet map” that range covers, if everything is spread out evenly in log space.

> **Why do this?**
>
> This is useful for comparing to a simple model where planets are equally likely at any size or distance (on a log scale), and for making fair comparisons between different parts of your data.

In [16]:
def integrate_loguniform(a_low,a_high,m_low,m_high):
    """
    Integrate a log-uniform distribution over the given range.
    
    Parameters
    ----------
    a_low, a_high : float
        Bounds on semi-major axis in au.
    m_low, m_high : float
        Bounds on planet mass in Earth masses.
    
    Returns
    -------
    float
        Area in ``log10(a)``-``log10(m)`` space.
    """
    return (np.log10(a_high)-np.log10(a_low)) * (np.log10(m_high)-np.log10(m_low))

### What does `calc_weights` do?

This function helps you figure out how important each event (like each planet you found) is, by giving it a “weight.”  
These weights help you count things fairly, taking into account how likely you were to find each event and how common they should be.

#### How does it work?

- It uses a math formula (an “integral”) to figure out the total area you’re looking at.
- For each event, it calculates a special weight (using the `cassan` formula and other factors).
- It multiplies the original weight for each event by these new factors to get the final weight.

> **Why do this?**
>
> These weights make sure your results are fair and accurate, so you don’t over-count or under-count certain types of events.

In [17]:
def calc_weights(df,event_rate_weight=1,int_func=integrate_loguniform,raww_col='final_weight'):
    """
    Calculate final event weights using a survey model.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Table of events.
    event_rate_weight : float, optional
        Overall scaling applied to all events.
    int_func : callable, optional
        Function used to integrate the assumed planet distribution.
    raww_col : str, optional
        Column in ``df`` containing the raw event weights.
    
    Returns
    -------
    ndarray
        Array of updated weights.
    """
    #get the cassan integral                                                                                                                                                                                           
    int_value = int_func(a_low,a_high,a_low,a_high)
    #for all events, calculate the cassan weight                                                                                                                                                                       
    weight = cassan(data)
    new_weights = df[raww_col]*cassan(data)*int_value*event_rate_weight
    return new_weights

### What does `get_2dhist_values` do?

This function helps you figure out, for each planet you found, how easy it was for your survey to detect it, based on its size and distance from its star.

#### How does it work?

- It looks at each planet’s mass and distance.
- It finds which “box” (bin) on your survey’s sensitivity map matches that planet.
- It grabs the sensitivity value from that box—this tells you how likely your survey was to find a planet like that.
- It does this for every planet in your list and gives you back all the sensitivity values.

> **Why do this?**
>
> Knowing the sensitivity for each planet helps you correct your results, so you don’t miss out on planets that were hard to find or over-count ones that were easy to spot.

In [18]:
def get_2dhist_values(grid,sample,a_bins,m_bins):
    """
    Extract survey sensitivity for each planet in a sample.
    
    Parameters
    ----------
    grid : ndarray
        Two-dimensional survey sensitivity grid.
    sample : pandas.DataFrame
        Planet sample.
    a_bins, m_bins : ndarray
        Bin edges defining the grid in log space.
    
    Returns
    -------
    list
        Sensitivity value for each planet.
    """
    mass_col = 'Planet_mass'
    a_col = 'Planet_semimajoraxis'
    a_bin_centers = ((a_bins[:-1]+a_bins[1:])/2)
    m_bin_centers = a_bin_centers
    mearth = 3.00374072e-6

    sensitivities = []

    for index, planet in sample.iterrows():
        #print('here')
        #print(planet,planet[mass_col])
        m_ind = np.abs(m_bin_centers - np.log10(planet[mass_col]/mearth)).argmin()
        a_ind = np.abs(a_bin_centers - np.log10(planet[a_col])).argmin()
        #XXXensure these are correctly indexed                                                                                                                                                                         
        sensitivities.append(grid[a_ind,m_ind])

    return sensitivities

### What does `apply_cuts` do?

This function helps you clean up your data by removing events (like planets) that don’t meet certain rules.  
It makes sure you only keep the events that are good enough for your analysis.

#### How does it work?

- It checks each event in your data to see if it passes several tests (called “cuts”):
  - Is the event’s signal strong enough?
  - Does it look enough like a real planet and not something else?
  - Is it in the right part of the sky and at the right time?
  - Does it pass other special checks?
- If an event fails any of these tests, it gets removed from your list.
- You get back a new list with only the good events.

> **Why do this?**
>
> This makes sure your results are based on reliable, high-quality data, so your conclusions are trustworthy.

In [19]:
def apply_cuts(dataFrame,flatchi2=500,chi2=160,u0max=3,t0inseason=True,noFS=False):
    """
    Apply quality cuts to a catalog of detected events.
    
    Parameters
    ----------
    dataFrame : pandas.DataFrame
        Input catalog of events.
    flatchi2 : float, optional
        Minimum acceptable improvement over a flat fit.
    chi2 : float, optional
        Minimum acceptable improvement over the single-lens fit.
    u0max : float, optional
        Maximum allowed impact parameter.
    t0inseason : bool, optional
        Require the event to occur within a season.
    noFS : bool, optional
        Remove events flagged as finite-source.
    
    Returns
    -------
    pandas.DataFrame
        Filtered catalog meeting all cuts.
    """
    dat = dataFrame.copy()
    # apply single lens chi2 cut - deafult is 500 above a flat light curve fit
    dat = dat[dat['ObsGroup_0_flatchi2']>=flatchi2]
    # apply binary lens cut - default is 160 above single lens fits
    dat = dat[dat['ObsGroup_0_chi2']>=chi2]
    # apply u0max cut- default is 3
    dat = dat[np.abs(dat['u0lens1'])<u0max]
    # apply inseasoncut
    if t0inseason:
        dat = dat[dat['ssn0']>0]
    # apply no finite source cut
    if noFS:
        dat=dat[dat['ObsGroup_0_FiniteSourceflag']==0]
    return dat

## Definitions

---

### What are these variables and what are they for?

These lines set up some basic numbers and grids that your analysis uses to organize and study planets:

- `mearth`: The mass of Earth, used as a reference to compare other planet masses.
- `a_low, a_high, m_low, m_high`: The lowest and highest values for planet distance from their star (semi-major axis, `a`) and planet mass (`m`). These set the boundaries for your study—only planets within these ranges are considered.
- `nbins`: The number of bins (sections) you want to split your mass and distance ranges into. More bins means finer detail, but can also make things noisier if you don’t have a lot of data.
- `m_bins`, `a_bins`: These are the edges of the bins for mass and distance, spaced evenly on a log scale. They define the grid you’ll use to count and analyze planets.
- `m_bin_centers`, `a_bin_centers`: The middle points of each bin, used for calculations and plotting.

#### Why might you want to change these?

- **Change the limits (`a_low`, `a_high`, `m_low`, `m_high`)** if you want to study a different range of planet sizes or distances.
- **Change `nbins`** if you want more or less detail in your analysis grid. More bins = more detail, but you need enough data to fill them.
- **Change `mearth`** only if you want to use a different reference mass (almost never needed).

In [20]:
# definitions   
# Mass of Earth 
mearth = 3.00374072e-6
# limits for sma and mass bins, needs to match injected planet distribution
a_low,a_high,m_low,m_high = 0.01,10.,0.01,10.
# number of bins  
nbins = 20
# make bin edges for mass-sma grid   
m_bins = np.linspace(np.log10(m_low),np.log10(m_high),nbins,endpoint=True)
a_bins = np.linspace(np.log10(a_low),np.log10(a_high),nbins,endpoint=True)
m_bin_centers = (m_bins[1:]+m_bins[:-1])/2  # tree?
a_bin_centers = (a_bins[1:]+a_bins[:-1])/2

## Data Set-Up, Weights, and Visualization

---

### Loading Your Planet Data

This cell loads all the planet events you want to analyze from a file.

**How to use this cell:**
- Make sure the `filename` variable points to the correct file you want to use. (It should be an HDF5 file with your planet data.)
- The `column_names` list tells the code which columns (data fields) to read from the file.
- When you run this cell, it reads the data and puts it into a table called `tot` that you can use for the rest of your analysis.

> **Tip:**  
> If you want to use a different data file, just change the `filename` path to point to your new file.

In [ ]:
# read in all events
column_names = ['Lens_Mbol', 't0lens1', 'tcroin', 'u0lens1', 'ObsGroup_0_chi2', 'ObsGroup_0_flatchi2', 'ObsGroup_0_FiniteSourceflag', 'Lens_b', 'Lens_l', 'Field',
                'Planet_semimajoraxis', 'Planet_mass', 'Planet_q',
                'final_weight'
               ]
#filename = '/fs/project/gaudi.1/crisp/gulls/romanc7_hz_simple/analysis/romanc7_hz_simple.out.hdf5'
filename = '/fs/project/gaudi.1/crisp/gulls/romanc7_hz_simple/analysis.real/romanc7_hz_simple.out.hdf5'
tot = pd.read_hdf(filename, columns=column_names)

### Adding Extra Information to Your Data (Insolation)

This cell adds two new columns to your planet data to help with your analysis:

- **`Planet_insolation`**:  
  This calculates how much energy (light) each planet gets from its star, based on the star’s brightness and the planet’s distance.  
  (Planets closer to bright stars get more energy; this is important for figuring out things like habitability.)

- **`ssn0`**:  
  This checks if the event (when the planet was found) happened during the “season” when your survey was actually watching.  
  (It uses the [`ssn`](#what-does-ssn-do) function to decide if the timing was right.)

> **Why do this?**  
>
> Adding these columns gives you more information about each planet, so you can make better cuts, plots, or calculations later in your analysis.

In [ ]:
# perform some calculations to append some columns that we will need
# first is insolation
tot['Planet_insolation'] = 10**(0.4*(4.74-tot['Lens_Mbol'])) * (tot['Planet_semimajoraxis'])**-2
# add a column indicating if t0 occurred in season
tot['ssn0'] = ssn(tot['t0lens1'])

### Applying the Survey Covering Factor and Final Weight Corrections

This section adjusts the weights for each event (planet) in your data to account for how much of the sky your survey actually covered, and applies some other corrections.

- **Covering Factor (`covfac`)**:  
  Not every part of the sky was observed equally. The “covering factor” tells you what fraction of each subfield (small patch of sky) was actually covered by your survey.  
  For each event, its weight is multiplied by the covering factor for its field, so you don’t over-count planets from areas you didn’t fully observe.

- **Final Weight Correction (`fwc`)**:  
  The code creates a new column, `final_weight`, that includes this covering factor correction (and could include other corrections, like for false positives using [`FPF`](#what-does-fpf-do)).

- **Survey Area Calculation**:  
  The code also adds up the total area covered by your survey, using the covering factors and the size of each subfield. This is a “sanity check” to make sure your survey area is what you expect.

> **Why do this?**  
>
> These corrections make sure your results are accurate and fair, so you don’t overestimate how common planets are by forgetting about parts of the sky you didn’t actually observe.

**Example outputs:**

```
0       l  -5.0
1       b  -4.0
2  covfac   0.0
1.8931712961709124
```

> Note:
>
> That `scalefac` variable is supposed to normalize the event rates to something physically meaningful, like "one planet per star per decade of mass," but Samson couldn't be bothered to explain how. For now, leaving it as scalefac = 1 is fine for validating the pipeline, but before you can get the absolute correct occurrence rate, you need to know what that number is supposed to be. It's Samson's homework, but ask Matt.

In [ ]:
#### Apply covering factor ####
# This goes through and multiplies each event for the fraction of the subfield it was drawn from
# that is covered by a given survey footprint.
# Also apply corr for log-uniform planet injection
tot['fwc'] = tot['final_weight']
# Also, there is a false positive fraction (FPF) that can be accounted for from Penny 2019
#dat['final_weight'] *= (1 - FPF(dat,mearth))
covfac = pd.read_csv('/home/crisp.92/Programs/gulls/scripts/covfac/layout7f.approx.covfac',index_col='ID_src')
# Running sum of the covering factor, adds together each unique covfac number and multiplies by
# the subfield size to calculate the area of the survey footprint. Just a sanity check. 
cov_sum = 0
# for each unique field of events
for field in np.unique(tot['Field']):
    # generate a mask for events that were drawn from that field
    mask = tot['Field'] == field
    # Essentially just renormalizes the event rates to be "one per star per mass per SMA bin"
    scalefac = 1  # !!!
    # This creates a new final weight corrected (fwc) column
    tot.loc[mask,'final_weight'] = scalefac * covfac.loc[field]['covfac'] * tot.loc[mask]['final_weight']
    # update running covfac sum
    cov_sum += covfac.loc[field]['covfac']*.2*.2
    # print number of events, the field number, and the covfac value
    #print(tot.loc[mask,'final_weight'].shape,field,covfac.loc[field]['covfac'])
# print survey area from covfac calc
print(cov_sum)

### Filtering Events with Detection Cuts

This section filters your data to create two new tables of events, based on how strict the detection rules (“cuts”) are:

- **`dat`**:  
  This table includes all events that pass the basic “in season” and signal strength checks, but with a very loose (almost no) cut on the “chi2” value.  
  (It uses the [`apply_cuts`](#what-does-apply_cuts-do) function.)

- **`det`**:  
  This table includes only the events that pass both the “in season” check and a stricter “chi2” cut, meaning they are more likely to be real planet detections.

> **Why do this?**  
>
> By making two filtered lists, you can compare all possible events to the ones that are most likely real planets, which helps you understand your survey’s performance and the reliability of your detections.

#### A Note on Filtering

As per Ali's notes, this step creates our two essential dataframes:
* **`dat`**: Represents all events that could have been detected (similar to `N_dat`), passing a basic `flatchi2 > 500` cut.
* **`det`**: Represents the events that were *actually* detected (our `N_det`), passing the stricter `chi2 > 160` cut.

These cuts are essential for correctly calculating the detection efficiency later.

In [7]:
# filter tot for only events with t0 in season
# dat is essentially single lens detected events with t0inSeason
dat = apply_cuts(tot,flatchi2=500, chi2=-1e9)
# det is detected planets with t0inSeason
det = apply_cuts(tot,flatchi2=500, chi2=160)

### Plotting the Survey Field Extents

This section makes a quick plot to show where your events are on the sky.

- It prints the minimum and maximum values of `Lens_b` (latitude) and `Lens_l` (longitude) for your events, so you can see the range your survey covers.
- It adjusts any longitude values greater than 300 by subtracting 360, to keep all longitudes in a standard range (useful for plotting).
- It then makes a scatter plot of longitude vs. latitude for all your events, so you can visualize the area of the sky your survey looked at.

> **Why do this?**  
>
> This plot gives you a quick check that your data covers the sky as expected, and helps you spot any weird outliers or mistakes in the event positions.

**Example Output:**  

```
-2.2057731 -0.39435861
5.2688221e-09 360.0
```

Below is an example of what this plot should look like:

![Example survey field plot](ExampleFigures/Figure1.png)

In [ ]:
# Just make a plot of field extents more less, not important
print(np.min(dat['Lens_b']),np.max(dat['Lens_b']))
print(np.min(dat['Lens_l']),np.max(dat['Lens_l']))
dat.loc[dat['Lens_l']>300,'Lens_l'] = dat[dat['Lens_l']>300]['Lens_l'] - 360
plt.plot(dat['Lens_l'],dat['Lens_b'],'.')

### Plotting the Distribution of Planet Distances

This section makes a histogram (bar chart) showing how many planets you have at different distances from their stars (measured as the semi-major axis, or SMA).

- It takes the logarithm of each planet’s distance to spread out the values more evenly (since distances can vary a lot).
- It counts how many planets fall into each distance bin and makes a plot.

> **Why do this?** 
> 
> This plot helps you see where most of your planets are—are they mostly close to their stars, or far away? It’s a quick way to understand the range and distribution of your data.

**Example Output:**  
Below is an example of what this plot should look like:

![Example SMA distribution plot](ExampleFigures/Figure2.png)

In [ ]:
# plot the distribution of SMA
plt.hist(np.log10(tot['Planet_semimajoraxis']),bins=200)#,weights=det['fwc'])
plt.show()

### Plotting the Distribution of Event Times

This section makes two histograms showing when your events happened, using two different time measurements (`t0lens1` and `tcroin`).

- It counts how many events occurred at each time, using the final weights (`fwc`) to make the counts fair.
- Both histograms are shown together, so you can compare the timing of the two types of events.

> **Why do this?** 
> 
> This plot helps you see if your events are spread out evenly over time, or if there are certain periods when more events were detected. It can also help you spot any gaps or patterns in your data collection.

**Example Output:**  
Below is an example of what this plot should look like:

![Example event time distribution plot](ExampleFigures/Figure3.png)

In [ ]:
plt.hist(tot['t0lens1'],range=[0,2010],bins=200,weights=tot['fwc'])
plt.hist(tot['tcroin'],range=[0,2010],bins=200,weights=tot['fwc'])
plt.show()

### Plotting the Distribution of Impact Parameters

This section makes a histogram showing the distribution of `u0lens1`, which is the impact parameter for each event (a measure of how close the event was to the center of the lensing effect).

- It counts how many events have each value of `u0lens1`, using the final weights (`fwc`) to make the counts fair.
- The range is set from -5 to 5, and the data is split into 200 bins.

> **Why do this?**  
>
> This plot helps you see if your events are mostly close to the center (small `u0lens1`) or more spread out. It’s useful for checking if your data matches what you’d expect from your survey and models.

**Example Output:**  
Below is an example of what this plot should look like:

![Example impact parameter distribution plot](ExampleFigures/Figure4.png)

In [ ]:
plt.hist(tot['u0lens1'],range=[-5,5],bins=200,weights=tot['fwc'])
plt.show()

### Calculating the Total Number of Corrected Events

This section calculates the total number of events in your filtered data, after applying all the corrections and weights.

- It adds up the `final_weight` for all events in your filtered table (`dat`).
- The result is printed out as the total “corrected” number of events.

> **Why do this?**  
>
> This gives you a fair count of how many events you really have, taking into account all the adjustments for survey coverage, detection efficiency, and other corrections.

**Example Output:**  
```
311680.162493718675
```

In [ ]:
# Get the total number of events
corrected_total_events = np.sum(dat['final_weight'])
print(corrected_total_events)

## Detection Efficiency

### Building 2D Histograms for Mass and Distance

This section creates two 2D histograms (tables) that count how many events you have for each combination of planet mass and distance from their star (semi-major axis).

- **`dat_eff`**:  
  Counts all filtered events, using their weighted values, and sorts them into bins based on the logarithm of their distance and mass.

- **`det_eff`**:  
  Does the same thing, but only for the events that passed the stricter detection cuts (likely real planets).

> **Why do this?**  
>
> These 2D histograms let you see how your events are distributed across both mass and distance at the same time.  
They are useful for making heatmaps or for further analysis of how detection efficiency changes with planet properties.

In [13]:
# build histograms for mass-SMA
dat_eff = np.histogram2d(np.log10(dat['Planet_semimajoraxis']),np.log10(dat['Planet_mass']/mearth),bins=[a_bins,m_bins],
                          weights=dat['fwc'])     
det_eff = np.histogram2d(np.log10(det['Planet_semimajoraxis']),np.log10(det['Planet_mass']/mearth),bins=[a_bins,m_bins],
                          weights=det['fwc'])     

### Plotting the Detection Efficiency Map

This section calculates and visualizes how good your survey is at detecting planets, depending on their mass and distance from their star (semi-major axis).

- It computes the detection efficiency (`det_efficiency`) for each bin by dividing the number of detected events by the total number of possible events.
- It then calculates the **Survey Sensitivity (`S`)**, defined as `S = det_efficiency * N_events` [(Suzuki et al. 2016)](https://ui.adsabs.harvard.edu/abs/2016ApJ...833..145S/abstract). This tells you how many planets you would have detected in that bin if every single star had a planet.
- It multiplies this efficiency by the total number of corrected events to get the survey sensitivity.
- It then makes a heatmap plot, where color shows how easy or hard it is to detect planets in each part of the mass–distance grid.

> **Why do this?**  
>
> This plot helps you see where your survey is most sensitive (where you’re most likely to find planets) and where it’s less sensitive. It’s a key tool for understanding and correcting for biases in your results.

**Example Output:**  
Below is an example of what this plot should look like:

![Example detection efficiency map](ExampleFigures/Figure5.png)

In [ ]:
# compute the detection sensitivity in terms of mass-SMA. 
det_efficiency = det_eff[0]/dat_eff[0] 
survey_sensitivity = det_efficiency * corrected_total_events
if True:        
    fig,ax = plt.subplots()
    phist = np.log10(det_efficiency).T
    #phist = np.log10(det_eff).T
    phist = np.clip(phist,-4,None)
    im = ax.imshow(phist,origin='lower',extent=[np.log10(a_low),np.log10(a_high),np.log10(m_low),np.log10(m_high)],vmin=-4,vmax=-1,cmap='cividis')
    #ax.scatter(np.log10(planet_sample['Planet_semimajoraxis']),np.log10(planet_sample['Planet_mass']/mearth),c=np.log10(planet_sample['sensitivity']/corrected_total_events),vmin=-4,vmax=-1,cmap='cividis')
    ax.set_xlabel(r'Semi-Major Axis [$\log(a/a_\oplus)$]',fontsize=16)
    ax.set_ylabel(r'Planet mass [$\log(M_p/M_\oplus)$]',fontsize=16)
    #plt.colorbar(im,label=r'$\log (N_{det})$ if each lens has one planet at $(M,a)$')
    cb = plt.colorbar(im,label=r'$\log ({Detection Efficiency})$')
    cb.set_label(label=r'$\log ({Detection Efficiency})$', fontsize=16)
    ax.tick_params(axis='both', which='major', labelsize=12)
    cb.ax.tick_params(labelsize=12)
    plt.savefig('ma_map.pdf')

### Plotting the Detection Efficiency Map in Mass–Insolation Space

This section calculates and visualizes how good your survey is at detecting planets, depending on their mass and the amount of energy (insolation) they receive from their star.

- It creates 2D histograms for both all events and detected events, sorted by planet mass and insolation.
- It computes the detection efficiency for each combination of mass and insolation by dividing the number of detected events by the total number of possible events in each bin.
- It multiplies this efficiency by the total number of corrected events to get the survey sensitivity.
- It then makes a heatmap plot, where color shows how easy or hard it is to detect planets in each part of the mass–insolation grid.

> **Why do this?**  
>
> This plot helps you see where your survey is most sensitive to finding planets, not just by their mass and distance, but also by how much energy they get from their star—a key factor for habitability studies.

**Example Output:**  
Below is an example of what this plot should look like:

![Example detection efficiency in mass–insolation space](ExampleFigures/Figure6.png)

In [ ]:
# build histograms for mass-insolation
s_low, s_high = 0.01, 10
s_bins = np.linspace(np.log10(s_low),np.log10(s_high),nbins,endpoint=True)
dat_eff_insol = np.histogram2d(np.log10(dat['Planet_insolation']),np.log10(dat['Planet_mass']/mearth),bins=[s_bins,m_bins],weights=dat['final_weight'])     
det_eff_insol = np.histogram2d(np.log10(det['Planet_insolation']),np.log10(det['Planet_mass']/mearth),bins=[s_bins,m_bins],weights=det['final_weight'])#/det_uniform[:,50]*3)#/det_uniform[:,50])

# detection efficiency in mass-insolation space     
efficiency_insol = det_eff_insol[0]/dat_eff_insol[0]
# survey sensitivity in mass-SMA space   
s_survey_sensitivity = efficiency_insol * corrected_total_events

s_bin_centers = (s_bins[1:]+s_bins[:-1])/2
m_bin_centers = (m_bins[1:]+m_bins[:-1])/2
test = 1#occ_arr(true_parameters,a_bin_centers,m_bin_centers)
fig,ax = plt.subplots()
phist = np.log10(efficiency_insol).T#[:,::-1]
phist = np.clip(phist,-5,None)
im = ax.imshow(phist,origin='lower',extent=[np.log10(s_high),np.log10(s_low),np.log10(m_low),np.log10(m_high)],vmin=-4,vmax=-1,cmap='cividis')
#ax.scatter(np.log10(planet_sample[:,43]),np.log10(planet_sample[:,42]/mearth),c=np.log10(planet_sample[:,0]),vmin=-4,vmax=2,cmap='cividis')  
ax.set_xlabel(r'Insolation [$\log(S/S_\oplus)$]',fontsize=16)
ax.set_ylabel(r'Planet mass [$\log(M_p/M_\oplus)$]',fontsize=16)
cb = plt.colorbar(im,label=r'$\log ({Detection Efficiency})$')
cb.set_label(label=r'$\log ({Detection Efficiency})$', fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=12)
cb.ax.tick_params(labelsize=12)
plt.savefig('ms_map.pdf')


### Counting Detected Planets by Mass

This section creates a histogram that counts how many detected planets you have in each mass bin.

- It takes the logarithm of each detected planet’s mass (relative to Earth) and sorts them into bins.
- It uses the final weights (`fwc`) to make the counts fair.
- The result is an array of counts—one for each mass bin.

> **Why do this?**  
>
> This gives you a quick summary of how your detected planets are distributed by mass, which is useful for understanding your sample and for further statistical analysis.

**Example Output:**  
```
array([ 0.05476764, 0.27029432, 0.10063701, 0.11163519, 0.15077024,
0.33513537, 0.7068122 , 0.83286857, 1.43670399, 1.84264488,
3.35712462, 4.2943674 , 5.78544942, 7.77331711, 11.30759032,
13.99050355, 19.20021137, 23.36225181, 30.17258706])
```

In [ ]:
det_hist = np.histogram(np.log10(det['Planet_mass']/mearth),bins=m_bins,weights=det['fwc'])
det_hist[0]

### Plotting the Detected Planet Mass Distribution

This section makes a line plot showing how many detected planets you have in each mass bin.

- It plots the center of each mass bin on the x-axis (in log scale, relative to Earth’s mass).
- It plots the logarithm of the number of detected planets in each bin on the y-axis.
- The plot includes grid lines for easier reading.

> **Why do this?**  
>
> This plot helps you quickly see which planet masses are most common in your detected sample, and whether there are trends or gaps in your data.

**Example Output:**  
Below is an example of what this plot should look like:

![Example detected planet mass distribution plot](ExampleFigures/Figure7.png)

In [ ]:
plt.plot(m_bin_centers,np.log10(det_hist[0]),label='Detections')
plt.xlabel(r"$\log(M_p/M_\oplus)$")
plt.grid()

## Survey Sensitivity

### Plotting Survey Sensitivity by Planet Mass

This section visualizes how sensitive your survey is to detecting planets of different masses.

- It integrates the survey sensitivity over all distances for each mass bin, showing how likely you are to detect a planet of each mass.
- It plots a line for each mass bin, colored sequentially, to show the sensitivity pattern.
- The thick black line shows the total sensitivity summed over all bins.
- The y-axis is on a logarithmic scale to show a wide range of values.

> **Why do this?**  
>
> This plot helps you see which planet masses your survey is best at detecting, and where it might be missing planets due to lower sensitivity.

**Example Output:**  
Below is an example of what this plot should look like:

![Example survey sensitivity by mass plot](ExampleFigures/Figure8.png)

```
Total Events  31680.162493718675
```

In [ ]:
cmap = plt.get_cmap('viridis')
mass_sens = scipy.integrate.simpson(survey_sensitivity,dx=a_bins[1]-a_bins[0],axis=0)
tot_sens = scipy.integrate.simpson(mass_sens,dx=a_bins[1]-a_bins[0],axis=0)
print(tot_sens)

fig, ax = plt.subplots()

# Plot lines with sequential colors
for i, y in enumerate(m_bin_centers):
    ax.plot(m_bin_centers, survey_sensitivity[i,:], color=cmap(i / len(m_bin_centers)))
    
ax.plot(m_bin_centers,mass_sens,linewidth=5)

plt.ylabel('Ndet per mass bin')
plt.yscale('log')
plt.xlabel('log(m)')
plt.show()
print('Total Events ', corrected_total_events)

## MLE Occurance Rate Fit Preparation

---

### Setting Up True Model Parameters for Testing

This section defines the “true” parameters for your planet occurrence rate model.  
These are the values you’ll use as a reference or for testing your fitting code.

- It makes copies of your filtered data (`dat_real` and `det_real`) to keep the originals safe.
- It sets up two sets of model parameters:
  - **Full model:** Uses a broken power law with different slopes and break points for planet mass.
  - **Simplified model:** Uses a single power law for planet mass and distance.
- The parameters are stored in dictionaries (`true_parameters` and `true_params`) for easy use in later calculations or fitting routines.

> **Why do this?**  
>
> Defining these “true” parameters lets you test your analysis pipeline, check if your code can recover known values, and compare different models.  
> It’s a key step for validating your results and understanding how your model works.

#### Note on the Model Being Fit

Right now, we're fitting a simple power law for mass (`n`) and semi-major axis (`m`):

$$ \frac{d^2N}{d\log(m_p) d\log(a)} = C \cdot m_p^n \cdot a^m $$

The ultimate goal is to implement and fit a more complex, scientifically-motivated model, like the modified Cassan model from [Penny (2019)](https://ui.adsabs.harvard.edu/abs/2019ApJS..241....3P/abstract), which uses a broken power law for the mass distribution. For now, this simpler model is good for testing the pipeline.

In [20]:
#make copies just in case   
dat_real = dat.copy()
det_real = det.copy()

#for full functional fitting of broken power law with varying break point in mass
#define injected occurrence rate parameters    
logC1_true = np.log10(2)
logMS1_true = 1
n1_true = 0
#logMbreak_true = np.log10(5.2)     
Mbreak_true = (5.2)
logC2_true = np.log10(2)
logMS2_true = np.log10(95)
n2_true = -0.73
m_true = 0

#set up as a dictionary    
true_parameters = {'logC1':logC1_true,
                   'logMS1':logMS1_true,
                   'n1':n1_true,
                   'logMbreak':np.log10(Mbreak_true),
                   'logC2':logC2_true,
                   'logMS2':logMS2_true,
                   'n2':n2_true,
                   'm':m_true}

#simplified version 
n_true = -1
m_true =0
logC_true = np.log10(2)    

#set up as a dictionary    
true_params = {'logC':logC_true,
               'n':n_true,
               'm':m_true}

### Drawing a Sample of Detected Planets for Analysis

This section prepares a sample of detected planets for further analysis, using your model and survey corrections.

- It calculates a new weight (`fwc`) for each detected planet, combining the final weight, a normalization factor, and the value from your occurrence rate model (using [`mass_func_arr`](#what-does-mass_func_arr-do)).
- It prints the expected total number of detections based on these weights.
- It draws a random sample of planets from your detected events, using the new weights to make the selection fair (using [`draw_sample`](#what-does-draw_sample-do)).
- It copies the sample for safety and then calculates the detection sensitivity for each planet in the sample (using [`get_2dhist_values`](#what-does-get_2dhist_values-do)).
- It prints the actual number of planets in the sample.

> **Why do this?**  
>
> This process gives you a realistic set of detected planets, accounting for all the corrections and your model, so you can use them for statistical analysis or further modeling.

**Example Output:**  

```
Expected total detections: 953.78942559785
Actual sample size: 968
```

In [ ]:
### draw planet sample    
#using cassan weights, currently in -2 (or -8 for uctc altered file)      
# new: where is -2 coming from???     
#new weights = raww * 3 * 3 * (occurrence rate value) 
#the 3's come from normalizing to the injected distribution of planets   
det['fwc'] = 9*det['final_weight']*mass_func_arr(true_params,det)
print("Expected total detections:",np.sum(det['fwc']))
#draw a sample          
planet_temp = draw_sample(det,weight_col='fwc')
#copy just in case? Not sure  
planet_sample = planet_temp.copy()
#retrieve the individual senstivity values for the sample of planets      
planet_sample['sensitivity'] =  get_2dhist_values(survey_sensitivity,planet_sample,a_bins,m_bins)
print("Actual sample size:", planet_sample.shape[0])

### Checking the Normalization of Detection Weights

This section checks that your detection weights and calculations are consistent.

- It calculates the ratio of the sum of the new weights (`fwc`) to the sum of the original final weights for detected planets.
- It prints the sum of the expected detections using two different methods, to make sure they match:
  - Directly multiplying the final weights by the occurrence rate model and normalization factor.
  - Summing the already-calculated `fwc` values.

> **Why do this?**  
>
> This is a quick consistency check to make sure your weights and calculations are correct. If the printed numbers match, you know your normalization is working as expected.

**Example Output:**  

```
953.78942559785
953.78942559785
```

In [ ]:
np.sum(det['fwc'])/np.sum(det['final_weight'])
print(np.sum(9*det['final_weight']*mass_func_arr(true_params,det)))
print(np.sum(det['fwc']))

## "Modelling" the Occurance Rates

---

### Fitting the Occurrence Rate Model to Your Sample

This section fits your planet occurrence rate model to the sample of detected planets, using maximum likelihood estimation (MLE).

- It sets up the model parameters (like normalization, mass slope, and distance slope) and their allowed ranges.
- It uses the [`lmfit`](https://lmfit.github.io/lmfit-py/) library to find the best-fit parameters that maximize the likelihood (using [`ln_like_neg`](#what-does-ln_like_neg-do)).
- After fitting, it calculates the expected number of detected planets using the best-fit model and the survey sensitivity.
- It integrates the occurrence rates over the grid to get the total expected number of detections, and prints the result.

> **Why do this?**  
>
> This step finds the model that best matches your data, allowing you to estimate how common different types of planets are, given your survey’s sensitivity.

**Example Output:**  

```
N_exp from fit: 968.0690398100909 968.0690398100909
```

In [ ]:
## fitting the sample  
params = lmfit.Parameters()
n_value = 0

#some more or less random values at this point. Boundaries are fairly lenient.
# boolean for each paramater determines if that parameter is varied 
params.add_many(('logC',np.log10(1),True,0,3),
                ('n',n_value,True,-5,5),
                ('m',0,True,-5,5))

# run the MLE fit 
#survey_sensitivity = np.nan_to_num(survey_sensitivity,nan=0)
#res = lmfit.minimize(ln_like_neg, params, args=(planet_sample,det_real),method='Nelder')#,bounds=bounds)#,callback=callbackF) 
#res = lmfit.minimize(ln_like_neg, params, args=(planet_sample,survey_sensitivity),method='Nelder')#,bounds=bounds)#,callback=callbackF) 
res = lmfit.minimize(ln_like_neg, params, args=(planet_sample,survey_sensitivity),method='Nelder')#,bounds=bounds)#,callback=callbackF) 

# essentially formatting   
occ_arr_res = occ_arr(res.params,a_bin_centers,m_bin_centers)
det_yield = (occ_arr_res*survey_sensitivity)
#print(occ_arr_res)
# integrate the occurrence rates  
#int1 = scipy.integrate.simpson([scipy.integrate.simpson(zz_x,x=a_bin_centers) for zz_x in occ_arr_res.T*survey_sensitivity],x=m_bin_centers)
inner_int = np.array([scipy.integrate.simpson(row,x=a_bin_centers) for row in det_yield])
outer_int = scipy.integrate.simpson(inner_int,x=m_bin_centers)
int1 = N_exp_real(res.params, planet_sample, survey_sensitivity)
print("N_exp from fit: ", outer_int, int1)

### Printing the Best-Fit Model Parameters

This line prints out the values of your best-fit model parameters after fitting:

- `10**res.params['logC']`: The normalization constant for your occurrence rate model (converted from log scale to normal scale).
- `res.params['n'].value`: The slope of the occurrence rate with respect to planet mass.
- `res.params['m'].value`: The slope of the occurrence rate with respect to semi-major axis (distance).

> **Why do this?**  
>
> Printing these values tells you the actual numbers your model found to best match your data.  
You can use these to interpret your results, compare to other studies, or use them in further calculations.

**Example Output:**  

In [ ]:
print(10**res.params['logC'], res.params['n'].value, res.params['m'].value)

## Occurance Rate Fitting Results

---

### Comparing Observed and Model Planet Mass Distributions

This section creates a plot that compares the observed distribution of planet masses to the predictions from your occurrence rate model.

- It shows several lines:
  - The raw (uncorrected) counts of planets per mass bin.
  - The corrected counts, accounting for survey sensitivity and other factors.
  - The “true” occurrence rate used for testing (if available).
  - The best-fit occurrence rate from your model.
- The y-axis is on a logarithmic scale to show a wide range of values.
- The plot includes a legend to explain each line.

> **Why do this?**  
>
> This plot lets you see how well your model matches the data, after all corrections. It’s a key check for the quality of your fit and the reliability of your results.

**Example Output:**  
Below is an example of what this plot should look like:

![Example planet mass distribution comparison plot](ExampleFigures/Figure9.png)

In [ ]:
# survey_sensitivity integrated over semi-major axis (a) 
mass_sens = scipy.integrate.simpson(survey_sensitivity,dx=a_bins[1]-a_bins[0],axis=0)
#mass_sens = scipy.integrate.simpson(det_efficiency.T,dx=a_bins[1]-a_bins[0],axis=1)

# plot the results of the LME     
m_bin_centers = (m_bins[:-1]+m_bins[1:])/2
# build mass histograms, relevant vars  
hist,bins = np.histogram(np.log10(planet_sample['Planet_mass']/mearth),bins=m_bins)
hist_true,bins = np.histogram(np.log10(det['Planet_mass']/mearth),bins=m_bins,weights=det['fwc'])

# histograms of full sim results  
# THIS LINE WAS RECENTLY HAD LAST COMMENT ADDED 
massw,bins = np.histogram(np.log10(dat['Planet_mass']/mearth),bins=m_bins,weights=dat['fwc'])
massd = np.histogram(np.log10(det['Planet_mass']/mearth),bins=m_bins,weights=det['fwc'])[0]

# I don't think this is needed
sma_sens = np.histogram(np.log10(det['Planet_semimajoraxis']),bins=a_bins,weights=det['fwc'])[0]/np.histogram(np.log10(dat['Planet_semimajoraxis']),bins=a_bins,weights=dat['fwc'])[0]
m_points = np.linspace(-2,1,1000)  

# plot mass
fig,ax = plt.subplots()

# could be needed for normalization, need to actually plug in real integral form
sma_int = 1 #(a_high**res.params['m'] - a_low**res.params['m']) / res.params['m'] / np.log(10) / 3
print(sma_int)


# plot uncorrected counts (units are counts per bin width)   
ax.semilogy(m_bin_centers, hist/(m_bins[1]-m_bins[0]),'C0--',label='Raw Sample/bin')#*(m_bins[1]-m_bins[0]))                                                                                                  
# plot true counts per bin width     
ax.semilogy(m_bin_centers,( hist_true/(m_bins[1]-m_bins[0])),'C6--',label='Raw Counts/bin')#*(m_bins[1]-m_bins[0]))
# calculate the corrected corrected observed counts  
corr = hist  / mass_sens / sma_int / (m_bins[1]-m_bins[0]) #/corr_fac                                                                                                                                               
# calculate the true injected mass function  
true_x = mass_func_arr(true_params,m_bin_centers)
# calculate the corrected true counts                                                                                                                                                                     
corr1 = hist_true / mass_sens / sma_int / (m_bins[1]-m_bins[0]) #/ corr_fac                                                                                                                                         
# plot true counts                   
ax.semilogy(m_bin_centers, corr,'C1',label='Corrected Sample')
ax.semilogy(m_bin_centers, corr1,'C0',label='Corrected Counts')
# Plot the true mass function   
ax.semilogy(m_points,mass_func_arr(true_params,m_points),'C1--',linewidth=5,label='True Rate')
ax.semilogy(m_points,mass_func_arr(res.params,m_points),'C2',label='Fit Rate')
#ax.semilogy(m_bin_centers,2/(hist_true/(m_bins[1]-m_bins[0])),'k',label='Required for Corr')
# plot mass sens
#ax.semilogy(m_bin_centers, 1/mass_sens,'C4--',label='1/Mass Sens')
# plot fitted function
#ax.semilogy(m_points,mass_func_plot(res.params,m_points),'C2',label='Fit')
ax.set_xlabel(r'$log(M/M_\oplus)$')
ax.set_ylabel(r'$dN/dlogM$')
plt.legend()
#ax.semilogy(m_points,mass_func(start,m_points))  
#ax.semilogy(m_points,mass_func(res.x,m_points)) 
plt.show()

### Comparing Observed and Model Semi-Major Axis Distributions

This section creates a plot that compares the observed distribution of planet semi-major axes (distances from their stars) to the predictions from your occurrence rate model.

- It shows several lines:
  - The raw (uncorrected) counts of planets per semi-major axis bin.
  - The corrected counts, accounting for survey sensitivity and other factors.
  - The “true” occurrence rate used for testing (if available).
  - The best-fit occurrence rate from your model.
- The y-axis is on a logarithmic scale to show a wide range of values.
- The plot includes a legend to explain each line.

> **Why do this?**  
>
> This plot lets you see how well your model matches the data for planet distances, after all corrections. It’s a key check for the quality of your fit and the reliability of your results.

**Example Output:**  
Below is an example of what this plot should look like:

![Example semi-major axis distribution comparison plot](ExampleFigures/Figure10.png)

In [ ]:
# this should be analagous plot for SMA distribution, some testing and figuring needs to be done still
# first of all should not be using mass_func_arr(), need a new version
# maybe should be evaluating for fixed mass or SMA?

# survey_sensitivity integrated over semi-major axis (a) 
sma_sens = scipy.integrate.simpson(survey_sensitivity.T,dx=m_bins[1]-m_bins[0],axis=0)
#mass_sens = scipy.integrate.simpson(det_efficiency.T,dx=a_bins[1]-a_bins[0],axis=1)

# plot the results of the LME     
a_bin_centers = (a_bins[:-1]+a_bins[1:])/2
# build mass histograms, relevant vars  
hist,bins = np.histogram(np.log10(planet_sample['Planet_semimajoraxis']),bins=a_bins)
hist_true,bins = np.histogram(np.log10(det['Planet_semimajoraxis']),bins=a_bins,weights=det['fwc'])

# histograms of full sim results  
# THIS LINE WAS RECENTLY HAD LAST COMMENT ADDED 
smaw, bins = np.histogram(np.log10(dat['Planet_semimajoraxis']),bins=a_bins,weights=dat['fwc'])
smad, bins = np.histogram(np.log10(det['Planet_semimajoraxis']),bins=a_bins,weights=det['fwc'])

#plot x points
sma_points = np.linspace(-2,1,1000)  

# plot mass
fig,ax = plt.subplots()



# plot uncorrected counts (units are counts per bin width)   
ax.semilogy(a_bin_centers, hist/(a_bins[1]-a_bins[0]),'C0--',label='Raw Sample/bin')#*(m_bins[1]-m_bins[0]))                                                                                                  
# plot true counts per bin width     
ax.semilogy(a_bin_centers,( hist_true/(a_bins[1]-a_bins[0])),'C6--',label='Raw Counts/bin')#*(m_bins[1]-m_bins[0]))
# calculate the corrected corrected observed counts  
corr = hist  / sma_sens / (a_bins[1]-a_bins[0]) #/corr_fac                                                                                                                                               
# calculate the true injected mass function  
true_x = mass_func_arr(true_params,a_bin_centers)
# calculate the corrected true counts                                                                                                                                                                     
corr1 = hist_true / sma_sens / (a_bins[1]-a_bins[0]) #/ corr_fac                                                                                                                                         
# plot true counts                   
ax.semilogy(a_bin_centers, corr,'C1',label='Corrected Sample')
ax.semilogy(a_bin_centers, corr1,'C0',label='Corrected Counts')
# Plot the true mass function   
ax.semilogy(sma_points,mass_func_arr(true_params,sma_points),'C1--',linewidth=5,label='True Rate')
ax.semilogy(sma_points,mass_func_arr(res.params,sma_points),'C2',label='Fit Rate')
#ax.semilogy(m_bin_centers,2/(hist_true/(m_bins[1]-m_bins[0])),'k',label='Required for Corr')
# plot mass sens
#ax.semilogy(m_bin_centers, 1/mass_sens,'C4--',label='1/Mass Sens')
# plot fitted function
#ax.semilogy(m_points,mass_func_plot(res.params,m_points),'C2',label='Fit')
ax.set_xlabel(r'$log(a/AU)$')
ax.set_ylabel(r'$dN/dloga$')
plt.legend()
#ax.semilogy(m_points,mass_func(start,m_points))  
#ax.semilogy(m_points,mass_func(res.x,m_points)) 
plt.show()

### Diagnostic Plots for Correction Factors

This section creates several diagnostic plots to help you understand and debug the correction factors used in your analysis.

- It plots different combinations of your mass sensitivity (`mass_sens`), corrected counts (`corr1`), and other related quantities.
- The lines show how these factors relate to each other across the mass bins.
- The y-axis is on a logarithmic scale, and the plot includes a legend for clarity.

> **Why do this?** 
> 
> These plots are mainly for troubleshooting and checking that your correction factors behave as expected.  
They help you spot any issues or inconsistencies in your sensitivity calculations or data corrections, making sure your final results are trustworthy.

**Tip:**  
You can use these plots to compare different ways of normalizing or correcting your data, and to ensure that your sensitivity and correction factors are working as intended.

**Example Output:**  
Below is an example of what this plot should look like:

![Example diagnostic correction factor plot](ExampleFigures/Figure11.png)



In [ ]:
#just some diagnostic plots when I was trying to figure out what was going on with corrections

#mass_sens = scipy.integrate.simpson(survey_sensitivity,dx=a_bins[1]-a_bins[0],axis=0)
fig,ax = plt.subplots()
ax.semilogy(m_bin_centers,2/corr1, label='Corr for hist/mass_sens')
ax.semilogy(m_bin_centers,mass_sens, label='mass_sens')
ax.semilogy(m_bin_centers,1/mass_sens,label='1/mass_sens')
ax.semilogy(m_bin_centers,2/( hist_true/(m_bins[1]-m_bins[0])),label='2/hist')
ax.set_xlabel(r'$log(M/M_\oplus)$')
ax.set_ylabel(r'$dN/dlogM$')
plt.legend()
#ax.semilogy(m_points,mass_func(start,m_points))  
#ax.semilogy(m_points,mass_func(res.x,m_points)) 
plt.show()

## Re-fitting with MCMC for Posterior Collecion

---

This is the part that tell us how good our estimate will actually be. 

### About MCMC

Markov Chain Monte Carlo (MCMC) is a powerful statistical technique used to explore the range of possible model parameters that fit your data.  
Instead of just finding the single “best” answer, MCMC helps you map out the full landscape of solutions, showing you the uncertainties and correlations between parameters.

**Why use MCMC?**
- It allows you to estimate not just the most likely values of your model parameters, but also their uncertainties and how they might be related.
- MCMC is especially useful when your model is complex or when the likelihood surface is not a simple shape.
- It provides a way to sample from the full “posterior distribution” of your parameters, giving you a much richer understanding of your results.

**In this notebook:**  
We use the [`emcee`](https://emcee.readthedocs.io/en/stable/) Python package, which is a popular and user-friendly tool for running MCMC analyses.

For more details, see the [emcee documentation](https://emcee.readthedocs.io/en/stable/).

### Setting Up the MCMC Sampler

This section sets up the parameters and starting positions for running a Markov Chain Monte Carlo (MCMC) analysis.

- It defines the number of walkers (`n_walkers`), the number of steps each walker will take (`n_steps`), and the number of initial steps to discard as “burn-in” (`n_burn`).
- It uses the results from your previous fit (`res.params`) as the starting point for the MCMC chains.
- It creates a set of slightly randomized starting positions for each walker, based on the best-fit values and a small spread (`starting_sigma`).
- It prepares variables for tracking the progress and convergence of the MCMC run (like `autocorr` and `old_tau`).

> **Why do this?**  
>
> MCMC is used to explore the range of model parameters that fit your data, giving you uncertainties and correlations.  
> Setting up the sampler with good starting points helps the chains converge faster and gives you more reliable results.

In [28]:
#### MCMC ####
# Was set up for iterative redraws of planet sample, keeping some of the framework for that
i = 0
filename = "mcmc_"+str(i)+".par4.nomaglim.invsens.h5"
n_walkers = 15
n_steps = 1000
n_burn = 100

#set-up
logC_start = res.params['logC']
n_start = res.params['n']
m_start = res.params['m']

starting_mean = np.array([logC_start,n_start,m_start])
starting_sigma = 0.01*np.ones(len(starting_mean))
starting = [(starting_mean + starting_sigma * np.random.randn(len(starting_mean))) for i in range(n_walkers)]

index = 0
autocorr = np.empty(n_steps)
old_tau = np.inf

### Initializing the MCMC Sampler and Backend

This section sets up the MCMC sampler and its backend for saving results.

- It prints a message to show the process is starting.
- It deletes any old MCMC result file with the same name, so you start fresh (remove this line if you want to continue or stack runs).
- It creates an HDF5 backend using `emcee`, which will save the MCMC samples to disk as the sampler runs.
- It sets up the sampler with the number of walkers, number of parameters, the probability function ([`ln_prob`](#what-does-ln_prob-do)), and the arguments your model needs.

**How to use this cell:**
1. Make sure the `filename` and folder (`data_penny/`) exist and are correct for your project.
2. If you want to keep results from previous runs, comment out or remove the `os.remove(...)` line.
3. Run this cell before starting the actual MCMC sampling loop.

> **Why do this?**  
>
> Saving the MCMC samples to disk lets you pause and resume runs, and makes it easy to analyze the results later without rerunning the whole chain.

In [ ]:
print('starting')

# delete old file, can be removed if you want to stack MCMC runs
os.remove("data_penny/"+filename)

backend = emcee.backends.HDFBackend("data_penny/"+filename)

n_parameters = len(starting_mean)

sampler = emcee.EnsembleSampler(n_walkers, n_parameters, ln_prob, args=(planet_sample, survey_sensitivity),backend=backend)

### Running the MCMC Sampling Loop and Checking for Convergence

This cell runs the MCMC sampler to explore the parameter space and checks for convergence as it goes.

- It starts a timer to measure how long the sampling takes.
- It runs the sampler for up to `n_steps` iterations, starting from the initial positions.
- Every 100 steps, it:
  - Prints the current iteration and checks the “autocorrelation time” (a measure of how independent the samples are).
  - Stores the average autocorrelation time for later analysis.
  - Checks if the chains have converged:
    - Convergence is assumed if the total number of steps is much larger than the autocorrelation time (at least 50 times), and if the autocorrelation time has stabilized (changed by less than 1%).
    - If converged, the loop breaks early to save time.
- The timer is stopped at the end to record how long the sampling took.

> **Why do this?**  
>
> This approach ensures you don’t waste time running the sampler longer than needed, and helps you get reliable, independent samples from the posterior distribution.  
> Monitoring the autocorrelation time is a standard way to check if your MCMC chains have “mixed” and are sampling the parameter space effectively.

In [ ]:
start = timeit.default_timer()
for sample in sampler.sample(starting, iterations=n_steps,progress=False):
    #print(sampler.iteration)
    # Only check convergence every 100 steps
    if sampler.iteration % 100:
        continue

    #ipdb.set_trace()
    print('Checking autocorr time at step: ',sampler.iteration)
    # Compute the autocorrelation time so far
    # Using tol=0 means that we'll always get an estimate even
    # if it isn't trustworthy
    tau = sampler.get_autocorr_time(tol=0)
    autocorr[index] = np.mean(tau)
    index += 1

    # Check convergence
    converged = np.all(tau * 50 < sampler.iteration)
    converged &= np.all(np.abs(old_tau - tau) / tau < 0.01)
    print('ACT at iter: '+str(autocorr[index-1]) +' at ' +str(int(sampler.iteration)))
    if converged:
        break
    old_tau = tau

stop = timeit.default_timer()

### MCMC Diagnostics

There really needs to be an additional cell here to plot the sampler chains.

* If walker trajectories (plot sampler.get_chain()) look like "hairy caterpillars," chains are mixed
* If they resemble "flat lines," chains are stuck (likely bad initialization or model mismatch)
* If they look like satifying overlapping scribbles with a consistent width, the sampler has converged and is effectively sampling the posterior.

In [ ]:
# MCMC Diagnostics

# [ ] plot the sampler chains

### Visualizing MCMC Results with a Corner Plot

This section visualizes the results of your MCMC run and shows how the model parameters are distributed and correlated.

- It prints a summary of how long the MCMC run took and how many samples were collected.
- It flattens the MCMC chains (removing the burn-in) to get a set of independent samples.
- It creates a “corner plot” using the [`corner`](https://corner.readthedocs.io/en/latest/) library, which shows:
  - The distribution (posterior) for each parameter (e.g., `logC`, `n`, `m`).
  - The correlations between parameters (e.g., how `n` and `logC` are related).
- It also plots many possible occurrence rate curves from the posterior, so you can see the range of models that fit your data.

> **Pro Tips for Using the Corner Plot:**
> - **Look for smooth, bell-shaped (Gaussian) distributions** for each parameter. This means your MCMC has likely converged and is sampling the true posterior.
> - **Check for correlations:** If you see slanted or curved shapes in the 2D plots (like between `n` and `logC`), it means those parameters are correlated—changing one affects the other.
> - **Watch for problems:** If you see weird shapes (multiple peaks, sharp edges, or “walls”), it could mean:
>   - The chains haven’t converged (try running longer).
>   - There are issues with your model or priors.
>   - The parameter space is not well-explored (try more walkers or a different starting point).

**Example Output:**  

```
Completed after  1000  samples in  0.011527339476160704  hours.
```

Below is an example of what a good corner plot should look like, with nice Gaussian posteriors and visible correlations:

![Example MCMC corner plot](ExampleFigures/Figure12.png)

**Example Model Curves:**  
The plot below shows many possible occurrence rate curves drawn from the MCMC posterior, illustrating the range of models that fit your data:

![Example model curves from MCMC](ExampleFigures/Figure13.png)

In [ ]:
#### Corner Plot ####

print('Completed after ', sampler.iteration, ' samples in ',(stop-start)/60/60 ,' hours.')
flat_samples = sampler.get_chain(discard=n_burn, flat=True)
# make sure this matches the labels of parameters you want to plot
labels = ['logC','n','m']#,'m']
fig = corner.corner(flat_samples, labels=labels, truths=starting_mean)
plt.show()

fig,ax = plt.subplots()
ax.semilogy(m_points,mass_func_arr(true_params,m_points),label='True',zorder=10000)
inds = np.random.randint(len(flat_samples), size=100)
for ind in inds:
    sample = flat_samples[ind]
    #print(sample)
    ax.semilogy(m_points,mass_func_arr(sample,m_points),'C1',alpha=0.1)
    #plt.plot(x0, np.dot(np.vander(x0, 2), sample[:2]), "C1", alpha=0.1)
plt.legend()
plt.show()

## How to Interpret the Results

Congratulations! You’ve worked through the full analysis pipeline. Here’s how to make sense of your results and what to do next:

### 1. **Best-Fit Model Parameters**
- The notebook provides the best-fit values for your occurrence rate model (e.g., normalization, mass slope, semi-major axis slope).
- **Interpretation:** These numbers tell you how common different types of planets are in your survey, and how occurrence rates change with planet mass and distance.

### 2. **Uncertainties and Correlations**
- The MCMC results (corner plots) show the range of parameter values that are consistent with your data, as well as how parameters are correlated.
- **Interpretation:** Narrow, bell-shaped posteriors mean your data strongly constrain the parameters. Wide or oddly-shaped posteriors mean more uncertainty or possible model issues.
- **Correlations** (slanted ellipses in the corner plot) mean that changing one parameter can be compensated by changing another.

### 3. **Model vs. Data Plots**
- The comparison plots show how well your model matches the observed distributions of planet mass and semi-major axis.
- **Interpretation:** If the “fit” and “true” lines closely follow the corrected data, your model is a good description of the observed population.

### 4. **Survey Sensitivity**
- The sensitivity maps and summary plots show where your survey is most and least sensitive to finding planets.
- **Interpretation:** Be cautious about drawing conclusions in regions where your sensitivity is low—your survey may have missed planets there.

### 5. **Physical Meaning**
- Use the best-fit parameters and their uncertainties to answer scientific questions, such as:
  - How common are Earth-mass planets at 1 AU?
  - Does planet occurrence increase or decrease with mass or distance?
  - Are your results consistent with previous studies?

### 6. **Caveats and Next Steps**
- **Check for convergence:** Make sure your MCMC chains have mixed and your posteriors look reasonable.
- **Compare to example outputs:** If your results look very different, double-check your data and settings.
- **Consider model limitations:** If your model doesn’t fit well, try a different functional form or add more parameters.
- **Calculate η_Earth:** Use your final, fitted occurrence rate model to calculate the actual fraction of stars hosting Earth-like planets. This involves integrating your model `d²N / dmdS` over a defined "Earth-like" box in mass and insolation space.

---

**In summary:**  
Your results provide a quantitative, uncertainty-aware measurement of planet occurrence rates in your survey.  
Use these findings to inform further research, compare with other surveys, or refine your models.

If you have questions or want to try new models or data, you can always revisit earlier steps in the notebook.

---

## Boss Battle: Fitting the Right Model
Okay, time for the final check. The goal here is to prove the notebook can correctly find the "true" parameters you used to create your simulation. But this only works if you fit the same kind of model that you simulated.

So, first, the most important question: What kind of model did you use for your simulation?

### Path A: If you simulated a SIMPLE Power Law...

Then congratulations, you've already won. The main part of the notebook, as it is, is designed to fit a simple power law.

Your "Boss Battle" was just running the MCMC and confirming that the final plots show the "Fit Rate" perfectly matching your "True Rate." If they line up and your corner plot looks good, you have successfully validated the pipeline for a simple model. The uncertainties you got are the RST-science-requirement secret sauce. You're done. Go have a drink.

### Path B: If you simulated a BROKEN Power Law...

Okay, now it's a real fight. Your simulation is more complex, and you need to prove the notebook can handle it. This is how you start getting results that look like the ones in the [Suzuki et al. (2012)](https://ui.adsabs.harvard.edu/abs/2016ApJ...833..145S/abstract) and [Penny et al. (2019)](https://ui.adsabs.harvard.edu/abs/2019ApJS..241....3P/abstract) papers.

#### Your Mission:

1.  **Activate the Complex Parameters:** In the **"Setting Up True Model Parameters for Testing"** cell, you need to switch from the simple model to the full one.
    * **Comment out** the `simplified version` of the parameters.
    * **Uncomment** the "full functional fitting" `true_parameters` dictionary. It has more pieces, like a `logMbreak` and different slopes (`n1`, `n2`). Make sure these values match the simulation you're trying to test.

2.  **Upgrade Your Fitting Functions:** The simple functions won't work with this new model. You will need to modify them to handle a broken power law.
    * **In `mass_func_arr`**: You need to change the logic to use one slope (`n1`) for planets *below* the break mass and another slope (`n2`) for planets *above* it. It will look something like this conceptually:
        ```python
        # Conceptual change for mass_func_arr
        if M_planet < M_break:
            # Use the first slope
            rate = C1 * (M_planet / M_ref1)**n1
        else:
            # Use the second slope
            rate = C2 * (M_planet / M_ref2)**n2
        ```
    * **In `ln_prior`**: You have to update the priors to check the allowed ranges for all the new parameters in your `true_parameters` dictionary (`logMbreak`, `n1`, `n2`, etc.).

3.  **Adjust the MCMC Setup**: In the **"Setting Up the MCMC Sampler"** cell, you'll need to update the `starting_mean` array and the `labels` for the corner plot to include all the new parameters you're fitting.

This is a significant step up, so don't be surprised if it takes a few tries. But getting this to work is how you go from just testing the code to doing real science with it. Good luck.

### Path C: The data are real...

So, you're done with simulations and are using actual, real-sky survey data. This is no longer a test; this is for all the marbles. The goal now isn't to recover a known truth, but to *discover* it.

**Your Mission:**

1.  **There Are No "True" Parameters:** The `true_params` dictionary is now irrelevant. You can comment it out or delete it entirely. The MCMC fit you get at the end *is* the result. The "truths" you were plotting in the corner plot don't exist anymore.

2.  **Model Selection is Key:** Since you don't know the true shape of the planet distribution, you may need to test multiple models. You should probably:
    * Fit the **simple power-law model** first.
    * Then, fit the **broken power-law model** (like in Path B).

3.  **Compare the Models:** How do you know which model is better? You have to compare them. [The Suzuki et al. (2016)](https://ui.adsabs.harvard.edu/abs/2016ApJ...833..145S/abstract) paper does exactly this. They use a statistical test (the "Bayes factor") to show that the broken power-law model is a much better fit to their data than the simple one. You'll need to do a similar comparison to justify using a more complex model.

4.  **The MCMC Result IS The Answer:** The final corner plot and the parameter distributions it shows are your scientific result.
    * The **peak** of each distribution is your best measurement for that parameter.
    * The **width** of the distribution is your uncertainty, or your error bars.

When you're done, you won't be saying "my code recovered the right answer." You'll be saying, "we measured the slope of the planet mass function to be $n$ with an uncertainty of $x$."

This is the whole point.

You're right. My apologies. My explanation was too simplistic—you've got a sharper definition. It's not about *time*, it's about *space*. A subtle but crucial difference. Let's fix it.

It's sloppy to leave an imprecise explanation in there. We'll rewrite that whole appendix to be more exact.

Replace the previous appendix with this one. It's better.

***

## Appendix A: De-Mystifying the Math

If you're wondering what "detection efficiency" or "weights" are, you're in the right place. This is the boring-but-important stuff that makes the whole analysis work.

### What is Detection Efficiency?

In a perfect world, our telescope would see every single planet that exists in the survey fields. We don't live in a perfect world. The telescope has blind spots, it's better at finding big planets than small ones, and it's better at finding planets at certain distances.

**Detection Efficiency** is simply a measure of how good the survey is at finding a specific type of planet.

Think of it like this: Imagine you're looking for red marbles in a huge sandbox, but you're wearing sunglasses that make it harder to see the color red.
* In some parts of the sandbox with bright sunlight, your detection efficiency for red marbles is high.
* In shady parts, your detection efficiency is low.
* If you're also worse at spotting small marbles than big ones, your efficiency is lower for the small ones.

Your final "detection efficiency map" (like the ones in the notebook) is like a chart showing your chances of finding a marble of a certain size in a certain spot. We calculate it for the telescope using this simple formula:

$$\text{Detection Efficiency} = \frac{\text{Number of Planets We ACTUALLY Detected}}{\text{Total Number of Planets That EXISTED in the Survey Volume}}$$

This is the same as the `det_eff = N_det / N_dat` you see in the notes. We need this map to correct for all the planets the telescope inevitably missed, otherwise we'd get the wrong answer for how common planets are.

### How Do the Weights Work?

"Weight" is just a number that tells us how important or likely a single simulated event is. In this notebook, the weights get adjusted in a few steps.

1.  **The Base Weight (`final_weight` from the file):** Each event from the Gulls simulation comes with a starting weight. This number represents how likely that specific microlensing event is to happen in the universe, based on the simulation's Galactic model. A high weight means a more common type of event.

    ``` final_weight ∝ (probability of lens-source alignment) × (source star density) ```

2.  **The Geometric Correction (`covfac`):** The simulation generates events across a simple, square area of the sky. But the Roman telescope's camera has a specific footprint with gaps between the detectors. The **covering factor (`covfac`)** is the fraction of the simulated square that is *actually* covered by one of Roman's detectors. If the Roman footprint only covers 80% of that simulation area, the `covfac` is 0.8. We multiply the event's weight by this factor to correct for the part of the simulated area that was never even observed.

3.  **The Final Corrected Weight (`fwc`):** This is the most important weight for the analysis. It's the `Base Weight` × `Geometric Correction`. When we sum up the `fwc` of all the *detected* planets (`det`), it gives us our best estimate of the true number of planets that exist in that category. This is what we use to calculate the "Corrected Counts" in the final plots.

4.  **The Temporary Model Weight (for drawing a sample):** This one is confusing, but it's only used for the validation test. To create a realistic *mock* sample of planets to test the MCMC, we do one more multiplication:
    `fwc` × (value from the `true_params` occurrence rate model)

    This creates a temporary set of weights used *only* in the `draw_sample` function. The goal is to create a fake dataset of planets that we know follows a specific power law, so we can see if the MCMC can correctly recover the parameters of that law. Once the sample is drawn, these temporary weights are thrown away.